In [41]:
CENSUS_TRACTS_DIR = "census_tracts"
# Units are miles
STATIC_SQUARE_SIZE = 1
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6
LATITUDE_DELTA = STATIC_SQUARE_SIZE/LATITUDE_MILES
LONGITUDE_DELTA = STATIC_SQUARE_SIZE/LONGITUDE_MILES

In [49]:
import os
import xml.etree.ElementTree as ET
import collections
import json

In [21]:
# Read all census tracts and put points in dict
census_tracts_bounding = {}
census_tracts_points = {}
for fname in os.listdir(CENSUS_TRACTS_DIR):
    latitudes = []
    longitudes = []
    try:
        # Parse the XML file
        root = ET.parse("{}/{}".format(CENSUS_TRACTS_DIR, fname)).getroot()
        for type_tag in root.findall("node"):
            d = type_tag.attrib
            latitudes.append(float(d["lat"]))
            longitudes.append(float(d["lon"]))
        census_tracts_points[fname.strip(".txt")] = list(zip(latitudes, longitudes))
        census_tracts_bounding[fname.strip(".txt")] = [(min(latitudes), min(longitudes)), (max(latitudes), max(longitudes))]
    except:
        print(fname)

17_81.txt
9800.txt
3_07.txt
18_58.txt
19_19.txt
18_46.txt
23_12.txt
23_04.txt
24_30.txt
15_01.txt
2_05.txt
17_60.txt


In [43]:
def in_grid(grid_lat, grid_lon, lat, lon):
    return (grid_lat <= lat <= grid_lat + LATITUDE_DELTA and grid_lon <= lon <= grid_lon + LONGITUDE_DELTA)

In [44]:
census_tract_grids = collections.defaultdict(list)
for tract_name in census_tracts_rect:
    grid_lat = census_tracts_bounding[tract_name][0][0]
    grid_lon = census_tracts_bounding[tract_name][0][1]
    while grid_lat < census_tracts_bounding[tract_name][1][0]:
        while grid_lon < census_tracts_bounding[tract_name][1][1]:
            # if there exists a point in the osm file for this tract that is in this square, add this square for this tract
            for lat, lon in census_tracts_points[tract_name]:
                if in_grid(grid_lat, grid_lon, lat, lon):
                    census_tract_grids[tract_name].append([grid_lat, grid_lon])
                    break
            grid_lon += LONGITUDE_DELTA
        grid_lat += LATITUDE_DELTA

In [52]:
output_dict = {}
output_dict["GRID_SIZE"] = STATIC_SQUARE_SIZE
output_dict["LATITUDE_DELTA"] = LATITUDE_DELTA
output_dict["LONGITUDE_DELTA"] = LONGITUDE_DELTA
output_dict["census_tracts_bounding"] = census_tracts_bounding
output_dict["census_tract_grids"] = census_tract_grids

In [53]:
json.dump(output_dict, open("census_tracts_info.json", "w"))

In [ ]:
# To search
# Find all possible tracts it can be by searching through census_tracts_bounding
# Search through the possible tracts to see which square it falls under --> Now we 
# know which tract it is in and which grid it lies in so we can index into the time matrix